In [9]:

import torch
import cv2
from matplotlib.image import BboxImage
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
from PIL import Image
from src.data.constants import DATA_DIR, TIMEPOINTS, NB_DIR
from src.models.utils.model import get_instance_segmentation_model
from src.models.utils.utils import collate_fn
from torch.utils.data import DataLoader, Subset
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
from src.models.predict_model import get_model, get_mask, get_prediction
from src.models.BetaCellDataset import BetaCellDataset, get_transform
from tifffile.tifffile import TiffFile
from os.path import join
from os import listdir

In [11]:
# Set working directory to file location
dirr = '/home/gummz/dtu/cell/cell/notebooks'
os.chdir('/home/gummz/dtu/cell/cell/notebooks')


In [5]:
device = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Running on {device}.')
print(cv2.__version__)

Running on cpu.
4.5.5


In [17]:
# Get dataloaders
size = 1024
img_idx = 1700
time_str = '12_03_18H_29M_39S'
folder = f'interim/run_{time_str}'
# data_tr, data_val = get_dataloaders(resize=size)
dataset = BetaCellDataset(transforms=get_transform(train=False), resize=size)


In [50]:
test_data_root = '../../../cell_data/test/'
test_data_dir = 'patches/duct'


files = listdir(join(test_data_root, test_data_dir))
# Filter out already used files
file_paths = [join(test_data_root, test_data_dir, file) for file in files]
file_paths_tif = [file for file in file_paths if 'tif' in file]
file_paths_czi = [file for file in file_paths if 'czi' in file]
print(file_paths)


['../../../cell_data/test/patches/duct/duct_ts_LI-2019-04-11-emb5-pos1_tp157-A2B3_D2.tif', '../../../cell_data/test/patches/duct/duct_ts_LI-2019-02-05-emb5-pos2_tp191-A3A4B4C4_B3.tif', '../../../cell_data/test/patches/duct/duct_ts_LI-2019-04-11-emb8-pos3_tp245-A1A4B4_B3.tif', '../../../cell_data/test/patches/duct/duct_ts_LI-2018-11-20-emb6-pos1_tp104-D1_A1.tif', '../../../cell_data/test/patches/duct/duct_ts_LI-2019-06-13-emb2-pos1_tp204-B1C1D1_C4.tif', '../../../cell_data/test/patches/duct/duct_ts_LI-2019-02-05-emb5-pos3_tp210_A1.tif', '../../../cell_data/test/patches/duct/duct_ts_LI-2019-02-05-emb5-pos2_tp191-A3A4B4C4_D1.tif', '../../../cell_data/test/patches/duct/duct_ts_LI-2019-04-11-emb8-pos4_tp229-A1A2D3D4_D1.tif', '../../../cell_data/test/patches/duct/duct_ts_LI-2019-02-05-emb5-pos4_tp133-A4_C1.tif', '../../../cell_data/test/patches/duct/duct_ts_LI-2019-06-13-emb2-pos1_tp204-B1C1D1_D2.tif', '../../../cell_data/test/patches/duct/duct_ts_LI-2018-11-20-emb6-pos1_tp104-D1_C3.tif', '.

In [63]:

consecutive = False
file_path = file_paths_tif[6]
print(file_path)
with TiffFile(file_path) as f:
    pages = f.pages  # f.pages[1::2]
    n_img = len(pages)

    # Create sample indexes
    if consecutive:
        randint = np.random.randint(low=0, high=n_img - 9)
        idx = np.arange(randint, randint + 9)
    else:
        idx = sorted([int(i) for i in np.random.randint(0, n_img, 9)])
    
    print(pages[15].asarray().shape)
    sample_images = [pages[i].asarray() for i in idx]
    plt.figure(figsize=(10, 10))
    for i, (j, image) in enumerate(zip(idx, sample_images)):
        # Create plot of sample images
        plt.subplot(3, 3, i + 1)
        plt.imshow(image)
        plt.axis('off')
        plt.title(f'Page {j}', fontsize=18)

    title = f'Filename: {file_path.split("/")[-1]}\nPages: {len(f.pages)}'
    plt.suptitle(title, fontsize=24)
    # save = join(FIG_DIR, operation, f'{file}.jpg')
    plt.show()


/dtu-compute/tubes/raw_data/LI_2019-11-21_emb6_pos3.lsm


FileNotFoundError: [Errno 2] No such file or directory: '/dtu-compute/tubes/raw_data/LI_2019-11-21_emb6_pos3.lsm'